# LR Smoothing test analysis

The problem

* We observe an effect of unequal contribution of different users to the like rate of items
* Some users use "like" as next and have the average like rate of 0.9+
* Some users use "like" rarely for memes they really like. Such users may have like rates 0.1-0.3

The idea of the feature

* Lets create a smoothed like rate which will be greater if a meme is liked by users with the low average like rate and vise versa
* The details can be found in the [source code](https://github.com/ffmemes/ff-backend/blob/a281bb4b4ba8abe1eb6077f0c480cd0ee91885d1/src/stats/meme.py#L6)

Metrics

* Daily session length
* Retention (user returned the next day)
* Daily active users (users with one active reaction)

Statistical methods

* Bootstraping users

In [1]:
from datetime import datetime
import os

import pandas as pd
import polars as pl
import numpy as np
import psycopg2
from dotenv import load_dotenv

In [2]:
load_dotenv()
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)

In [3]:
q = """
select user_id, count(*) n_reactions, count(*) filter (where reaction_id = 1) n_likes, user_id % 100 >= 50 is_test, date_trunc('day', reacted_at) d
from user_meme_reaction
where reacted_at >= '2024-07-12 00:00:00'
and reacted_at < '2024-07-22 00:00:00'
and reaction_id is not null
group by user_id, d
"""

In [4]:
ab_results_df = pl.read_database(q, conn)

In [86]:
len(ab_results_df)

3969

In [6]:
users_test = ab_results_df.filter(pl.col('is_test') == True).select('user_id').unique()
users_control = ab_results_df.filter(pl.col('is_test') == False).select('user_id').unique()

In [59]:
def calc_dau(df):
    n_days = df.select((pl.max('d') - pl.min('d')).dt.total_days()).item() + 1
    return len(df) / n_days

def calc_n_reactions_p50(df):
    return df.select(pl.quantile('n_reactions', 0.5)).item()

def calc_n_reactions_p75(df):
    return df.select(pl.quantile('n_reactions', 0.75)).item()

def calc_retention(df):
    stage_1 = df.with_columns(next_d=pl.col('d') + pl.duration(days=1)).select('user_id', 'd', 'next_d')
    stage_2 = (
        stage_1
        .filter(pl.col('next_d') <= stage_1.select(pl.max('d')).item())
        .select('user_id', 'd')
        .join(stage_1, left_on=['user_id', 'd'], right_on=['user_id', 'next_d'], how='left')
    )
    return stage_2.select(pl.col('d_right').is_not_null().sum() / pl.len())

In [60]:
metrics = {
    'dau': calc_dau,
    'n_reactions_p50': calc_n_reactions_p50,
    'n_reactions_p75': calc_n_reactions_p75,
    'retention': calc_retention,
}
metrics_res = {'test': {}, 'control': {}}
for gr in ['test', 'control']:
    for m in metrics.keys():
        metrics_res[gr][m] = []

for idx in range(1000):
    for gr in ['test', 'control']:
        users = users_control
        if gr == 'test':
            users = users_test
        users_sampled = users.sample(n=len(users), shuffle=True, with_replacement=True, seed=idx+42)

        df = ab_results_df.join(users_sampled, on='user_id', how='inner')
        for m in metrics.keys():
            metrics_res[gr][m].append(metrics[m](df))

In [66]:
metrics_agg = []
for m in metrics.keys():
    for gr in ['test', 'control']:
        metrics_agg.append({
            'group': gr,
            'metric': m,
            'mean': np.mean(metrics_res[gr][m]),
            'std': np.std(metrics_res[gr][m]),
        })
metrics_agg = pl.DataFrame(metrics_agg)

In [81]:
metrics_diff = []
for m in metrics.keys():
    diff = np.array(metrics_res['test'][m]) / np.array(metrics_res['control'][m])
    metrics_diff.append({
        'metric': m,
        'diff_p05': np.percentile(diff, 5),
        'diff_p50': np.percentile(diff, 50),
        'diff_p95': np.percentile(diff, 95),
    })
metrics_diff = pl.DataFrame(metrics_diff)

In [83]:
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_float_precision(2)

polars.config.Config

In [84]:
print(metrics_agg)

shape: (8, 4)
┌─────────┬─────────────────┬────────┬──────┐
│ group   ┆ metric          ┆ mean   ┆ std  │
│ ---     ┆ ---             ┆ ---    ┆ ---  │
│ str     ┆ str             ┆ f64    ┆ f64  │
╞═════════╪═════════════════╪════════╪══════╡
│ test    ┆ dau             ┆ 208.96 ┆ 6.76 │
│ control ┆ dau             ┆ 187.82 ┆ 6.17 │
│ test    ┆ n_reactions_p50 ┆ 23.61  ┆ 2.18 │
│ control ┆ n_reactions_p50 ┆ 18.67  ┆ 1.45 │
│ test    ┆ n_reactions_p75 ┆ 83.50  ┆ 8.47 │
│ control ┆ n_reactions_p75 ┆ 64.74  ┆ 6.89 │
│ test    ┆ retention       ┆ 0.67   ┆ 0.02 │
│ control ┆ retention       ┆ 0.67   ┆ 0.02 │
└─────────┴─────────────────┴────────┴──────┘


In [85]:
print(metrics_diff)

shape: (4, 4)
┌─────────────────┬──────────┬──────────┬──────────┐
│ metric          ┆ diff_p05 ┆ diff_p50 ┆ diff_p95 │
│ ---             ┆ ---      ┆ ---      ┆ ---      │
│ str             ┆ f64      ┆ f64      ┆ f64      │
╞═════════════════╪══════════╪══════════╪══════════╡
│ dau             ┆ 1.03     ┆ 1.12     ┆ 1.20     │
│ n_reactions_p50 ┆ 1.05     ┆ 1.26     ┆ 1.53     │
│ n_reactions_p75 ┆ 1.01     ┆ 1.30     ┆ 1.64     │
│ retention       ┆ 0.93     ┆ 1.01     ┆ 1.09     │
└─────────────────┴──────────┴──────────┴──────────┘
